In [1]:
from config import gkey
import pandas as pd
import pycountry
import requests
import textract
import PyPDF2
import json
import re

### Initialize list sequence to store names of Countries

In [2]:
countries = [
    'Afghanistan','Albania','Algeria','Angola','Antigua and Barbuda',
    'Argentina','Armenia','Aruba','Australia','Austria','Azerbaijan',
    'The Bahamas','Bahrain','Bangladesh','Barbados','Belarus','Belgium', 
    'Belize','Benin','Bhutan','Bolivia','Bosnia and Herzegovina',
    'Botswana','Brazil','Brunei','Bulgaria','Burkina Faso','Burma',
    'Burundi','Cabo Verde','Cambodia','Cameroon','Canada','Central African Republic',
    'Chad','Chile','China','Colombia','Comoros','Congo, Democratic Republic of the',
    'Congo, Republic of the','Costa Rica',"Cote d'Ivoire",'Croatia','Cuba','Curaçao',
    'Cyprus','Czech Republic', 'Denmark','Djibouti', 'Dominican Republic','Ecuador',
    'Egypt','El Salvador','Equatorial Guinea', 'Eritrea','Estonia','Eswatini','Ethiopia',
    'Fiji', 'Finland', 'France','Gabon','The Gambia','Georgia','Germany','Ghana','Greece',
    'Guatemala','Guinea','Guinea-Bissau','Guyana','Haiti','Honduras','Hong Kong','Hungary',
    'Iceland','India','Indonesia','Iran','Iraq','Ireland','Israel','Italy','Jamaica','Japan',
    'Jordan','Kazakhstan','Kenya',"Korea, Democratic People's Republic of",'Korea, Republic of',
    'Kosovo','Kuwait','Kyrgyz Republic',"Laos",'Latvia','Lebanon','Lesotho','Liberia','Lithuania',
    'Luxembourg','Macau','Macedonia','Madagascar','Malawi','Malaysia','Maldives','Mali','Malta',
    'Marshall Islands','Mauritania','Mauritius','Mexico','Micronesia, Federated States of',
    'Moldova','Mongolia','Montenegro','Morocco','Mozambique','Namibia','Nepal','Netherlands',
    'New Zealand','Nicaragua','Niger','Nigeria','Norway','Oman','Pakistan','Palau','Panama',
    'Papua Guinea New','Paraguay','Peru','Philippines','Poland','Portugal','Qatar','Romania',
    'Russia','Rwanda','St. Lucia','St. Vincent and the Grenadines','Saudi Arabia','Senegal','Serbia',
    'Seychelles','Sierra Leone','Singapore','Slovakia','Slovenia','Solomon Islands','South Africa',
    'South Sudan','Spain','Sri Lanka','Sudan','Suriname','Sweden','Switzerland','Syria','Taiwan',
    'Tajikistan','Tanzania','Thailand','Timor-Leste','Togo','Tonga','Trinidad and Tobago','Tunisia',
    'Turkey','Turkmenistan','Uganda','Ukraine','United Arab Emirates','United Kingdom',
    'United States of America','Uruguay','Uzbekistan','Venezuela', 'Vietnam','Zambia','Zimbabwe',
    'Libya','St. Maarten','Somalia','Yemen' #Libya, St. Marteen, Somalia and Yemen are Special Cases
]


In [3]:
len(countries)

187

### Function to Parse the pdf file

In [4]:
def parse_file(filename): 
    # Read the file.
    pdfFileObj = open(filename,'rb')
    # The pdfReader variable is the readable object that will be parsed.
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    #Parse through all the pages.
    num_pages = pdfReader.numPages
    count = 0
    text = ""
    # The while loop will read each page.
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()
    # This if statement exists to check if the above library returned words. It's done because PyPDF2 cannot read scanned files.
    if text != "":
        text = text
    # If the above returns as False, run the OCR library textract to #convert scanned/image based PDF files into text.
    else:
        text = textract.process(fileurl, method='tesseract', language='eng')
    return text

text = parse_file('2018-TIP-Report.pdf')
    

### Function to Clean the parsed pdf

In [5]:
def clean_text(text):
    text = text.strip().replace("\n", "").replace("˜", "fi").replace("˚", "ft").replace("Sincerely,iiiiii","")\
                .replace("Trafiick","Traffick").replace("™", "'").replace("trafiick","traffick")\
                .replace('ﬁprevalence rateﬂ','"prevalence rate"').replace("˚ight","fight")\
                .replace("/", "").replace("  REPUBLIC OF THE:", " REPUBLIC OF THE:")\
                .replace("DEMOCRATIC PEOPLE'S  REPUBLIC OF", "DEMOCRATIC PEOPLE'S REPUBLIC OF")\
                .replace("PAPUA NEW GUINEA:", "PAPUA GUINEA NEW:").replace("⁄", "").replace("\\","")\
                .replace("ST. VINCENT AND  THE GRENADINES", "ST. VINCENT AND THE GRENADINES")\
                .replace("UNITED STATES  OF AMERICA", "UNITED STATES OF AMERICA")\
                .replace("461SPECIAL CASE: LIBYA", " ").replace("462SPECIAL CASE: LIBYA", " ")\
                .replace("463SPECIAL CASE: ST. MAARTEN", " ").replace("464SPECIAL CASE: SOMALIA", " ")\
                .replace("465SPECIAL CASE: SOMALIA", " ").replace("466SPECIAL CASE: YEMEN", " ")\
                .replace("467SPECIAL CASE: YEMEN", " ").replace("468SPECIAL CASE: YEMEN", " ")
    return text 

cleaned_text = clean_text(text)


### Function to Extract each country data from text

In [6]:
def extract_info(countries_list,text):
    # Create list to query info for each country
    countries_up = []
    for i in countries:
        if i in ["Libya", 'St. Maarten', 'Somalia', 'Yemen']:
            countries_up.append("SPECIAL CASE: "+i.upper())
        else: 
            countries_up.append(i.upper()+":")        
    new_list = []
    a = text.split(countries_up[0])
    for i in range(len(countries_up)-1):
        a = a[1].split(countries_up[i+1])
        new_list.append(a[0])
        if i+1 == len(countries_up)-1:
            new_list.append(a[1].split("469RELEVANT INTERNATIONAL CONVENTIONS")[0])
    return new_list

country_info = extract_info(countries,cleaned_text)
len(country_info)


187

### Function to Extract each country data from text

In [7]:
def extract_info(countries_list,text):
    # Create list to query info for each country
    countries_up = []
    for i in countries:
        if i in ["Libya", 'St. Maarten', 'Somalia', 'Yemen']:
            countries_up.append("SPECIAL CASE: "+i.upper())
        else: 
            countries_up.append(i.upper()+":")
    new_list = []
    a = text.split(countries_up[0])
    for i in range(len(countries_up)-1):
        a = a[1].split(countries_up[i+1])
        new_list.append(a[0])
        if i+1 == len(countries_up)-1:
            new_list.append(a[1].split("469RELEVANT INTERNATIONAL CONVENTIONS")[0])
    return new_list

country_info = extract_info(countries,cleaned_text)
len(country_info)


187

### Get all the coutntries data from text and extract countries that are special cases

In [8]:
country_dict = {i.upper():[{j:k}] for i,j,k in zip(countries, ["INTRO"]*187, country_info)}
special = [s.upper() for s in ['Libya','St. Maarten','Somalia','Yemen']]
special_country_dict = {s:country_dict[s] for s in special}
print(len(country_dict.values()))
print(len(special_country_dict))

187
4


### Function to remove cpecial cases countries from the other countries

In [9]:
def removekey(d,delete_items):
    r = dict(d)
    for key in delete_items: 
        del r[key]
    return r

normal_country_dict = removekey(country_dict, special)
print(len(normal_country_dict.values()))
normal_countr = list(normal_country_dict.keys())

183


### Next two functions creates a dictionary for each country

In [10]:
def append_data(country,i,data):
    data[0]["INTRO"] = data[0]["INTRO"].split(i)[1]
    data.append({"TIER":i})
    intro = data[0]["INTRO"].split(f"RECOMMENDATIONS FOR {country}")
    data[0]["INTRO"] = intro[0]
    recommend = intro[1].split("PROSECUTION")
    data.append({"RECOMMENDATIONS":recommend[0]})
    prosecute = recommend[1].split("PROTECTION")
    data.append({"PROSECUTIONS":prosecute[0]})
    protect = prosecute[1].split("PREVENTION")
    data.append({"PROTECTION":protect[0]})
    prevent = protect[1].split("TRAFFICKING PROFILE")
    data.append({"PREVENTION":prevent[0]})
    data.append({"TRAFFICKING PROFILE":prevent[1]})
    return data

In [11]:
def portion_data(country):
    data = normal_country_dict[country]
    tier = ["TIER 1", "TIER 2", "TIER 3"]
    for i in tier:
        length = len(data[0]["INTRO"].split(i))       
        if "TIER 2 WATCH LIST" in data[0]["INTRO"]:
            i = "TIER 2 WATCH LIST"
            append_data(country,i,data)
        elif length == 2 and "TIER 2 WATCH LIST" not in data[0]["INTRO"]: 
            append_data(country,i,data)            
    return data,country_dict


### Next 2 cells creates a list of dictionaries for countries with normal cases i.e Tier 1 - 3

In [12]:
new_normal_dict = {}
for i in normal_countr:
    try:
        new_normal_dict[i] = portion_data(i)
    except:
        pass

In [54]:
x = countries[:183]  #183
try:
    new_normal_dict = [{i.upper():portion_data(i.upper())[0]} for i in x]
except:
    pass

### Functions to create a dictionary for each special case country

In [58]:
def get_country_special(country):
    new_special = []
    intro = special_country_dict[country][0]["INTRO"].split("GOVERNMENT EFFORTS")
    new_special.append({"INTRO":intro[0]})
    new_special.append({"TIER": "SPECIAL CASE"})
    traffick = intro[1].split("TRAFFICKING PROFILE")
    new_special.append({"GOVERNMENT EFFORTS":traffick[0]})
    new_special.append({"TRAFFICKING PROFILE":traffick[1]})
    return new_special


### Create a list of dictionaries for countries with special cases

In [60]:
special = [i.upper() for i in ["Libya", 'St. Maarten', 'Somalia', 'Yemen']]
new_special_dict = [{i:get_country_special(i)} for i in special]

### Function to get trafficking profile from the list of dictionaries

In [16]:
def get_traffic_profile(country_list,country_dict):
    return [{a:list(country_dict[country_list.index(a)][a][-1].values())[0]} for a,b in zip(country_list, country_dict)]


### Get trafficking profile for both normal countries and special countries

In [17]:
normal_trafficking_profile = get_traffic_profile(normal_countr,new_normal_dict)
special_trafficking_profile = get_traffic_profile(special,new_special_dict)


### Next 2 cells adds both  profiles in a list to get profiles for all countries

In [18]:
special_trafficking_profile = get_traffic_profile(special,new_special_dict)
trafficking_profile = normal_trafficking_profile.copy()
trafficking_profile.extend(special_trafficking_profile)


In [19]:
new_normal_countr = normal_countr.copy()
new_normal_countr.extend(special)


### Corrected some spelling issues in the report

In [20]:
changed_countries = [
    'Afghanistan','Albania','Algeria','Angola','Antigua and Barbuda',
    'Argentina','Armenia','Aruba','Australia','Austria','Azerbaijan',
    'The Bahamas','Bahrain','Bangladesh','Barbados','Belarus','Belgium', 
    'Belize','Benin','Bhutan','Bolivia','Bosnia and Herzegovina',
    'Botswana','Brazil','Brunei','Bulgaria','Burkina Faso','Burma',
    'Burundi','Cabo Verde','Cambodia','Cameroon','Canada','Central African Republic',
    'Chad','Chile','China','Colombia','Comoros','Democratic',
    'Republic of The Congo','Costa Rica',"Cote d'Ivoire",'Croatia','Cuba','Curaçao',
    'Cyprus','Czech Republic', 'Denmark','Djibouti', 'Dominican Republic','Ecuador',
    'Egypt','El Salvador','Equatorial Guinea', 'Eritrea','Estonia','Eswatini','Ethiopia',
    'Fiji', 'Finland', 'France','Gabon','The Gambia','Georgia','Germany','Ghana','Greece',
    'Guatemala','Guinea','Guinea-Bissau','Guyana','Haiti','Honduras','Hong Kong','Hungary',
    'Iceland','India','Indonesia','Iran','Iraq','Ireland','Israel','Italy','Jamaica','Japan',
    'Jordan','Kazakhstan','Kenya',"North Korea",'South Korea',
    'Kosovo','Kuwait','Kyrgyz Republic',"Laos",'Latvia','Lebanon','Lesotho','Liberia','Lithuania',
    'Luxembourg','Macau','Macedonia','Madagascar','Malawi','Malaysia','Maldives','Mali','Malta',
    'Marshall Islands','Mauritania','Mauritius','Mexico','Micronesia','Moldova','Mongolia',
    'Montenegro','Morocco','Mozambique','Namibia','Nepal','Netherlands','New Zealand','Nicaragua',
    'Niger','Nigeria','Norway','Oman','Pakistan','Palau','Panama','Papua Guinea New','Paraguay',
    'Peru','Philippines','Poland','Portugal','Qatar','Romania','Russia','Rwanda','St. Lucia',
    'St. Vincent and the Grenadines','Saudi Arabia','Senegal','Serbia','Seychelles','Sierra Leone',
    'Singapore','Slovakia','Slovenia','Solomon Islands','South Africa','South Sudan','Spain',
    'Sri Lanka','Sudan','Suriname','Sweden','Switzerland','Syria','Taiwan','Tajikistan','Tanzania',
    'Thailand','Timor-Leste','Togo','Tonga','Trinidad and Tobago','Tunisia','Turkey','Turkmenistan',
    'Uganda','Ukraine','United Arab Emirates','United Kingdom','United States','Uruguay',
    'Uzbekistan','Venezuela', 'Vietnam','Zambia','Zimbabwe','Libya','St. Maarten','Somalia','Yemen' 
    #Libya, St. Marteen, Somalia and Yemen are Special Cases
]


### Function to get data frame showing base country and destinations and type of trafficking involved

In [21]:
def get_dataframe(countries,country_list,trafficking_profile):
    x = 0
    j = {
        "Base Country":[],
        "Relation Country":[],
        "Men": [],
        "Women":[],
        "Children":[],
        "Sex Trafficking":[],
        "Forced Labor":[],
        "Debt Bondage":[],
        "Description":[]
        }
    for country in country_list:
        a = list(trafficking_profile[x].values())[0].split(".")
        a = a[:len(a)-1]
        strip_country = countries.copy()
        d = strip_country.pop(country_list.index(country))   
        for b in a:
            for i in strip_country:
                if i in b:
                    if " men" in b or "women" in b or "boy" in b or "girl" in b or "child" in b or " Men" in b or "Women" in b or "Boy" in b or "Girl" in b or "Child" in b:
                        j["Relation Country"].append(i)
                        j["Base Country"].append(d)
                        j["Description"].append(b)
                        if " men" in b or " Men" in b or "adult" in b or "Adult" in b:
                                j["Men"].append(1)
                        else:
                            j["Men"].append(0)
                        if "women" in b or "Women" in b or "adult" in b or "Adult" in b:
                            j["Women"].append(1)
                        else:
                            j["Women"].append(0)
                        if "boy" in b or "girl" in b or "child" in b or "Boy" in b or "Girl" in b or "Child" in b:
                            j["Children"].append(1)
                        else:
                            j["Children"].append(0)
                        if "debt" in b or "Debt" in b:
                            j["Debt Bondage"].append(1)
                        else:
                            j["Debt Bondage"].append(0)
                        if "sex" in b or "Sex" in b:
                            j["Sex Trafficking"].append(1)
                        else:
                            j["Sex Trafficking"].append(0)
                        if "force" in b or "Forced" in b:
                            j["Forced Labor"].append(1)
                        else:
                            j["Forced Labor"].append(0)
        x += 1
    df = pd.DataFrame.from_dict(j, orient='index').transpose()
    return df

### Next 2 cells calls the function to get the data frame and sorts the data frame

In [22]:
trafficking_df = get_dataframe(changed_countries,new_normal_countr,trafficking_profile)

In [23]:
trafficking_df = trafficking_df.sort_values(by=['Base Country']).reset_index(drop=True)
trafficking_df.head()

,Base Country,Relation Country,Men,Women,Children,Sex Trafficking,Forced Labor,Debt Bondage,Description
0,Afghanistan,India,1,1,1,0,1,0,"Men, women, and children in Afghanistan often..."
1,Afghanistan,Tajikistan,0,1,1,1,0,0,There were reports of women and girls from th...
2,Afghanistan,Sri Lanka,0,1,1,1,0,0,There were reports of women and girls from th...
3,Afghanistan,Philippines,0,1,1,1,0,0,There were reports of women and girls from th...
4,Afghanistan,Pakistan,0,1,1,1,0,0,There were reports of women and girls from th...


### Next couple of cells gets the Lat and Lng of the base and relating countries and rearranges the columns

In [24]:
# Add columns for lat and lng of the base and relation countries
# Used "" to specify initial entry.
trafficking_df["Base Country Lat"] = ""
trafficking_df["Base Country Lng"] = ""
trafficking_df["Relation Country Lat"] = ""
trafficking_df["Relation Country Lng"] = ""
trafficking_df.head()

,Base Country,Relation Country,Men,Women,Children,Sex Trafficking,Forced Labor,Debt Bondage,Description,Base Country Lat,Base Country Lng,Relation Country Lat,Relation Country Lng
0,Afghanistan,India,1,1,1,0,1,0,"Men, women, and children in Afghanistan often...",,,,
1,Afghanistan,Tajikistan,0,1,1,1,0,0,There were reports of women and girls from th...,,,,
2,Afghanistan,Sri Lanka,0,1,1,1,0,0,There were reports of women and girls from th...,,,,
3,Afghanistan,Philippines,0,1,1,1,0,0,There were reports of women and girls from th...,,,,
4,Afghanistan,Pakistan,0,1,1,1,0,0,There were reports of women and girls from th...,,,,


### Function to get latitude and longitude of base and relation countries

In [25]:
def get_lat_lng(df,gkey,column,Lat,Lng):
    # Create a params dict that will be updated with new country each iteration
    params = {"key": gkey}
    # Loop through the df and run a lat/long search for each country
    for index, row in df.iterrows():
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"
        country = row[f'{column}']
        params['address'] = f"{country}"
        # Make request
        try:
            country_lat_lng = requests.get(base_url, params=params)
            # Convert to json
            country_lat_lng = country_lat_lng.json()
            df.loc[index, f"{Lat}"] = country_lat_lng["results"][0]["geometry"]["location"]["lat"]
            df.loc[index, f"{Lng}"] = country_lat_lng["results"][0]["geometry"]["location"]["lng"]
        except:
            df.loc[index, f"{Lat}"] = ""
            df.loc[index, f"{Lng}"] = ""
    return df


In [26]:
new_trafficking_df = get_lat_lng(trafficking_df,gkey,"Base Country","Base Country Lat","Base Country Lng")
new_trafficking_df = get_lat_lng(trafficking_df,gkey,"Relation Country","Relation Country Lat","Relation Country Lng")


In [27]:
new_trafficking_df = trafficking_df[["Base Country","Base Country Lat","Base Country Lng","Relation Country","Relation Country Lat","Relation Country Lng","Men","Women","Children","Sex Trafficking","Forced Labor","Debt Bondage","Description"]]


### Check for empty columns

In [28]:
null_columns=new_trafficking_df.columns[new_trafficking_df.isnull().any()]
new_trafficking_df[null_columns].isnull().sum()

Series([], dtype: float64)

In [46]:
new_trafficking_df#.head(20)

,Base Country,Base Country Lat,Base Country Lng,Relation Country,Relation Country Lat,Relation Country Lng,Men,Women,Children,Sex Trafficking,Forced Labor,Debt Bondage,Description
0,Afghanistan,33.9391,67.71,India,20.5937,78.9629,1,1,1,0,1,0,"Men, women, and children in Afghanistan often..."
1,Afghanistan,33.9391,67.71,Tajikistan,38.861,71.2761,0,1,1,1,0,0,There were reports of women and girls from th...
2,Afghanistan,33.9391,67.71,Sri Lanka,7.87305,80.7718,0,1,1,1,0,0,There were reports of women and girls from th...
3,Afghanistan,33.9391,67.71,Philippines,12.8797,121.774,0,1,1,1,0,0,There were reports of women and girls from th...
4,Afghanistan,33.9391,67.71,Pakistan,30.3753,69.3451,0,1,1,1,0,0,There were reports of women and girls from th...
5,Afghanistan,33.9391,67.71,Iran,32.4279,53.688,0,1,1,1,0,0,There were reports of women and girls from th...
6,Afghanistan,33.9391,67.71,Syria,34.8021,38.9968,0,0,1,0,0,0,"Reportedly, the Iranian government and the Is..."
7,Afghanistan,33.9391,67.71,Iran,32.4279,53.688,0,0,1,0,0,0,"Reportedly, the Iranian government and the Is..."
8,Afghanistan,33.9391,67.71,Greece,39.0742,21.8243,0,0,1,1,0,0,Some Afghan boys are subjected to sex traffic...
9,Afghanistan,33.9391,67.71,China,35.8617,104.195,0,1,1,1,0,0,There were reports of women and girls from th...


### Saving data as json

In [63]:
new_trafficking_df.to_json("Output/trafficking_report.json")
new_trafficking_df.to_csv("Output/trafficking_report.csv")

In [61]:
with open('Output/normal_cases.json', 'w') as fp:
    json.dump(new_normal_dict, fp)

## Save all the created data to Mongo Database

In [33]:
trafficking = trafficking_df.to_dict("list")

In [34]:
import pymongo
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Define database and collection
db = client.new_tip_hf_report_data
trafficking_collection = db.trafficking

### Function to insert created trafficking dataframes into mongodb

In [62]:
with open('Output/special_cases.json', 'w') as fp:
    json.dump(new_special_dict, fp)

In [35]:
def insert_one_into_mongodb(data_dict,collection):
    key = list(data_dict.keys())
    value = list(trafficking.values())
    try:
        collection.insert_one(data_dict)
    except pymongo.errors.DuplicateKeyError:
        pass
    return collection

### Insert trafficking dictionary

In [36]:
trafficking_collection = insert_one_into_mongodb(trafficking,trafficking_collection)


### Function to insert normal and special country list of dictionaries into Mongodb

In [37]:
def insert_many_into_mongodb(data_list,collection):
    for i in data_list:
        try:
            collection.insert(i,check_keys=False)
        except:
            pass
    return collection

### Insert normal cases dictionary

In [38]:
normal_collection = db.normal_cases

In [39]:
normal_cases_collection = insert_many_into_mongodb(new_normal_dict,normal_collection)


### Insert special cases dictionary

In [40]:
special_collection = db.special_cases

In [41]:
special_cases_collection = insert_many_into_mongodb(new_special_dict,special_collection)


### Function to show result in the Mongo data base

In [42]:
def show_mongo_result(collection):
    results = collection.find()
    for result in results:
        print(result)
    return

In [43]:
show_mongo_result(trafficking_collection)

{'_id': ObjectId('5e7ce2aae5c1129a7c773642'), 'Base Country': ['Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Albania', 'Albania', 'Algeria', 'Algeria', 'Algeria', 'Algeria', 'Algeria', 'Algeria', 'Algeria', 'Algeria', 'Algeria', 'Algeria', 'Angola', 'Angola', 'Angola', 'Angola', 'Angola', 'Angola', 'Angola', 'Angola', 'Angola', 'Angola', 'Angola', 'Antigua and Barbuda', 'Antigua and Barbuda', 'Antigua and Barbuda', 'Argentina', 'Argentina', 'Argentina', 'Argentina', 'Argentina', 'Argentina', 'Argentina', 'Argentina', 'Armenia', 'Armenia', 'Armenia', 'Aruba', 'Aruba', 'Aruba', 'Azerbaijan', 'Azerbaijan', 'Azerbaijan', 'Azerbaijan', 'Azerbaijan', 'Azerbaijan', 'Az

In [65]:
#show_mongo_result(normal_cases_collection)

In [64]:
#show_mongo_result(special_cases_collection)